In [1]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()
# db = FAISS.from_texts(texts=['9'], embedding=embeddings)
# db.save_local(folder_path=index_path, index_name=index_name) 
# db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [4]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Voy a sacar un pasaje para la familia de buenos aires a cordoba"
              },
              {
                "role": "system",
                "content": "quienes son los pasajeros?"
              },
              {
                "role": "user",
                "content": "Ana es mi hija mayor, tiene 20 años. Juan es mi hijo menor, tiene 10 años. Mi esposa se llama Maria y tiene 40 años. Yo me llamo Pedro y tengo 45 años"
              },
              {
                "role": "system",
                "content": "en que fecha quiere viajar?"
              },
              {
                "role": "user",
                "content": "del 20 de agosto al 30 de agosto"
              },
              {
                  "role": "system",
                  "content": "perfecto, salen el 20 de agosto a las 10:00 y vuelven el 30 de agosto a las 18:00. El precio total es de 1000 dolares. Desea confirmar la compra?"
              },
              {
                  "role": "user",
                  "content": "si"
              },
              {
                  "role": "system",
                  "content": "Gracias por su compra"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20200825T12:00:00.000Z",
          },
        },
        {
            "messages": [
                {
                "role": "user",
                "content": "Quiero subir de categoria en el programa de viajero frecuente"
                },
                {
                    "role": "system",
                    "content": "que categoria tiene actualmente?"
                },
                { 
                  "role": "user",
                  "content": "soy categoria 1"
                },
                {
                  "role": "system",
                  "content": "para subir de categoria debe tener 1000 millas. Cuantas millas tiene actualmente?"
                },
                {
                  "role": "user",
                  "content": "tengo 500 millas"
                },
                {
                  "role": "system",
                  "content": "puede comprar 500 millas por 100 dolares. Desea comprarlas?"
                },
                {
                  "role": "user",
                  "content": "si"
                },
                {
                  "role": "system",
                  "content": "Gracias por su compra. Ahora tiene 1000 millas y es categoria 2"
                }
            ],
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "context_id": "2",
                "created_at": "20220825T18:00:00.000Z",                
              }
        }
  ]


Se experimenta con un prompt resumidor de datos.

In [16]:
db = FAISS.from_texts(texts=['notebook-11'], embedding=embeddings)
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        # print(messages)
        # print(context['metadata'])
        extraction = gpt_system_user(
            system_message="""
              Extract every meaningful information about the user from the provided conversation.
              Ask yourself, what happened in the conversation?
              What data about the user did you learn?
              At what date and time was this data gathered?

              Create a summary of everything you learned about the user.



              """,
            user_message=f" METADATA: ```{context['metadata']} CONVERSATION: ```{messages}``` ```",
            model="gpt-3.5-turbo",
        )
        if extraction['status_code'] != 200:
            print(f"Error: {extraction['text']}")
            continue
        else:
            print(f"{extraction['text']['content']} \n\n\n///////////////////////////\n\n\n")
            db.add_texts(
                texts=[extraction['text']['content']], 
                metadatas=[context['metadata']],
            )

Summary:

The user, identified as Pedro, is planning to book a trip for his family from Buenos Aires to Cordoba. He mentions that his wife's name is Maria, he has a daughter named Ana who is 20 years old, and a son named Juan who is 10 years old. Pedro himself is 45 years old. 

Pedro wants to travel from August 20th to August 30th. The system confirms the travel dates and provides the departure and return times as well as the total price of $1000. Pedro confirms the purchase and thanks the system for the purchase. 

The data about the user gathered from this conversation includes the user's name (Pedro), the names and ages of his family members (Ana, Maria, and Juan), and the desired travel dates (August 20th to August 30th).

This data was gathered on August 25th, 2020 at 12:00:00 UTC. 


///////////////////////////



Summary:

From the conversation, we learned the following information about the user:

- The user wants to upgrade their status in the frequent traveler program.
- The

Preguntamos por la edad simplemente, para empezar, la respuesta deberia ser 23 si estamos en 2023.

In [19]:
user_input_3 = "que edad tiene mi hija mayor?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3)
answer_3["text"]["content"]


CONTEXT:{ message: Summary:

The user, identified as Pedro, is planning to book a trip for his family from Buenos Aires to Cordoba. He mentions that his wife's name is Maria, he has a daughter named Ana who is 20 years old, and a son named Juan who is 10 years old. Pedro himself is 45 years old. 

Pedro wants to travel from August 20th to August 30th. The system confirms the travel dates and provides the departure and return times as well as the total price of $1000. Pedro confirms the purchase and thanks the system for the purchase. 

The data about the user gathered from this conversation includes the user's name (Pedro), the names and ages of his family members (Ana, Maria, and Juan), and the desired travel dates (August 20th to August 30th).

This data was gathered on August 25th, 2020 at 12:00:00 UTC., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Según la información que tenemos en la conversación, tu hija mayor, Ana, tiene 20 años.'

Continúa el error.

Probamos exactamente el mismo prompt con gpt4.

In [20]:
user_input_3 = "que edad tiene mi hija mayor?"
filter = {"person_id": "20"}
db_results_3 = db.similarity_search_with_score(query=user_input_3, embeddings=embeddings, filter=filter, k=1)
context_3 = "{ message: " + db_results_3[0][0].page_content + ", metadata: " + str(db_results_3[0][0].metadata) + " }"
print(f"CONTEXT:{context_3}")
system_prompt_3 = f"""
You are a personal assistant. You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.
```{context_3}```

"""
answer_3 = gpt_system_user(system_message=system_prompt_3, user_message=user_input_3, model="gpt-4")
answer_3["text"]["content"]


CONTEXT:{ message: Summary:

The user, identified as Pedro, is planning to book a trip for his family from Buenos Aires to Cordoba. He mentions that his wife's name is Maria, he has a daughter named Ana who is 20 years old, and a son named Juan who is 10 years old. Pedro himself is 45 years old. 

Pedro wants to travel from August 20th to August 30th. The system confirms the travel dates and provides the departure and return times as well as the total price of $1000. Pedro confirms the purchase and thanks the system for the purchase. 

The data about the user gathered from this conversation includes the user's name (Pedro), the names and ages of his family members (Ana, Maria, and Juan), and the desired travel dates (August 20th to August 30th).

This data was gathered on August 25th, 2020 at 12:00:00 UTC., metadata: {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'} }


'Su hija Ana tenía 20 años en agosto de 2020. Dado que hoy es 13 de septiembre de 2023, Ana ahora debería tener 23 años.'

GPT razona mejor y resuelve el problema.